In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

### Test npys

In [2]:
path1 = "C:/Users/shant/Desktop/PICT x MIT/WASSA 2022/Ensamble/npys_of_ensemble_models/"
test_BERTBase_OUS_400 = np.load(path1 + 'test_BERTBase-OUS_400.npy')
test_ElectraBase_Augmented_Data_4528_maxlen = np.load(path1 + 'test_ElectraBase-Augmented_Data_4528_maxlen.npy')
test_ElectraBase_Augmented_Data_random_balanced_647 = np.load(path1 + 'test_ElectraBase-Augmented_Data_random_balanced_647.npy')
test_ElectraBase_OUS_400 = np.load(path1 + 'test_ElectraBase-OUS_400.npy')
test_ElectraBase_OUS_500 = np.load(path1 + 'test_ElectraBase-OUS-500.npy')

### Valid npys

In [3]:
path2 = "C:/Users/shant/Desktop/PICT x MIT/WASSA 2022/Ensamble/npys_of_ensemble_models/"
val_BERTBase_OUS_400 = np.load(path2 + 'val_BERTBase-OUS_400.npy')
val_ElectraBase_Augmented_Data_4528_maxlen = np.load(path2 + 'val_ElectraBase-Augmented_Data_4528_maxlen.npy')
val_ElectraBase_Augmented_Data_random_balanced_647 = np.load(path2 + 'val_ElectraBase-Augmented_Data_random_balanced_647.npy')
val_ElectraBase_OUS_400 = np.load(path2 + 'val_ElectraBase-OUS_400.npy')
val_ElectraBase_OUS_500 = np.load(path2 + 'val_ElectraBase-OUS-500.npy')

### Functions to convert emotions to indices and indices to emotions

In [4]:
e2i = {
            "anger": 0,
            "disgust": 1,
            "fear": 2,
            "joy": 3,
            "neutral": 4,
            "sadness": 5,
            "surprise": 6
        }
i2e = {v: k for k, v in e2i.items()}

def index_to_emotion(arr):
    i2e = {0: 'anger', 1: 'disgust', 2: 'fear', 3: 'joy', 4: 'neutral', 5: 'sadness', 6: 'surprise'}
    new_arr = []
    for i in range(len(arr)):
        new_arr.append(i2e[arr[i]])
    return new_arr

def emotion_to_index(arr):
    e2i = {'anger': 0, 'disgust': 1, 'fear': 2, 'joy': 3, 'neutral': 4, 'sadness': 5, 'surprise': 6}
    new_arr = []
    for i in range(len(arr)):
        new_arr.append(e2i[arr[i]])
    return new_arr

### y_true 

In [5]:
y_true = pd.read_csv('messages_dev_features_ready_for_WS_2022.tsv', sep = '\t')['emotion'].tolist()

### Ensembling Function
![Image of AOS AOUS](https://docs.google.com/drawings/d/e/2PACX-1vSjuA0grkvkwVHC_70GvTGzEoSp_RTRqgNQHJO6YQFaLlZYjMbfKDUshrM2WLA8NQppUskovPy_i7nd/pub?w=1047&h=571)

In [6]:
def get_outputs(models, model_scores, model_weights, len_predict):
    y_pred = [] 
    best_f1 = []
    
    for m in models:
        best_f1.append((model_scores[m],m))
    best_model = max(best_f1)[-1]  # Choosing model with highest f1 score (in case of clashes vote of this mode li prefered)
    
    for i in range(len_predict):
            
            a = [(models[model][i],model) for model in models]
            d = dict() # contains weighted votes {label: vote_counts}
            
            for j in a:
                if j[0] not in d:
                    d[j[0]] = model_weights[j[1]]
                else:
                    d[j[0]] += model_weights[j[1]]
            votes = [(d[k],k) for k in d]
            votes.sort()
            
            if votes[-1][0] == 1: # choosing the label with max votes
                y_pred.append(models[best_model][i])
            
            else: # if max votes = 1, choose the prediction of the best_model
                y_pred.append(votes[-1][-1])

    return y_pred

### Models {name: npys}

In [7]:
final_ensemble_models_val = {'val_BERTBase_OUS_400' : val_BERTBase_OUS_400,
          'val_ElectraBase_Augmented_Data_4528_maxlen' : val_ElectraBase_Augmented_Data_4528_maxlen, 
          'val_ElectraBase_Augmented_Data_random_balanced_647' : val_ElectraBase_Augmented_Data_random_balanced_647, 
          'val_ElectraBase_OUS_400' : val_ElectraBase_OUS_400}

final_ensemble_models_test = {'test_BERTBase_OUS_400' : test_BERTBase_OUS_400,
          'test_ElectraBase_Augmented_Data_4528_maxlen' : test_ElectraBase_Augmented_Data_4528_maxlen, 
          'test_ElectraBase_Augmented_Data_random_balanced_647' : test_ElectraBase_Augmented_Data_random_balanced_647, 
          'test_ElectraBase_OUS_400' : test_ElectraBase_OUS_400}

### Model f1 scores {name: f1 score}

In [8]:
final_ensemble_scores_val = {
 'val_BERTBase_OUS_400': 0.591857543223047,
 'val_ElectraBase_Augmented_Data_4528_maxlen': 0.5894212727171738,
 'val_ElectraBase_Augmented_Data_random_balanced_647': 0.5906288969076003,
 'val_ElectraBase_OUS_400': 0.5967307817451174,
}

final_ensemble_scores_test = {
 'test_BERTBase_OUS_400': 0.591857543223047,
 'test_ElectraBase_Augmented_Data_4528_maxlen': 0.5894212727171738,
 'test_ElectraBase_Augmented_Data_random_balanced_647': 0.5906288969076003,
 'test_ElectraBase_OUS_400': 0.5967307817451174,
}

### Model weights {name: npys}

In [9]:
model_weights_val = {
 'val_BERTBase_OUS_400': 1,
 'val_ElectraBase_Augmented_Data_4528_maxlen': 1,
 'val_ElectraBase_Augmented_Data_random_balanced_647': 1,
 'val_ElectraBase_OUS_400': 1,
}

model_weights_test = {
 'test_BERTBase_OUS_400': 1,
 'test_ElectraBase_Augmented_Data_4528_maxlen': 1,
 'test_ElectraBase_Augmented_Data_random_balanced_647': 1,
 'test_ElectraBase_OUS_400': 1,
}

### Prediction on Validation dataset, calculation of f1 score

In [10]:
y_pred_val = get_outputs(final_ensemble_models_val, final_ensemble_scores_val, model_weights_val, len(y_true))
print(f1_score(emotion_to_index(y_true), y_pred_val, average='macro'))

0.6276035369518614


In [30]:
model_outputs_dev = pd.read_csv(r'C:\Users\shant\Desktop\PICT x MIT\WASSA 2022\Paper Publication functions\model_outputs_dev.csv') #.to_dict()

In [37]:
model_outputs_dev['Ensemble_w1_pred'] = index_to_emotion(y_pred_val)

In [38]:
model_outputs_dev['Ensemble_w1_true'] = y_true

In [44]:
model_outputs_dev = model_outputs_dev[['essay', 'BERTBase_OUS_400_pred', 'BERTBase_OUS_400_true',
       'ElectraBase_Augmented_Data_4528_maxlen_pred',
       'ElectraBase_Augmented_Data_4528_maxlen_true',
       'ElectraBase_Augmented_Data_random_balanced_647_pred',
       'ElectraBase_Augmented_Data_random_balanced_647_true',
       'ElectraBase_OUS_400_pred', 'ElectraBase_OUS_400_true',
       'Ensemble_w1_true', 'Ensemble_w1_pred']]

In [46]:
model_outputs_dev.to_csv(r'C:\Users\shant\Desktop\PICT x MIT\WASSA 2022\Paper Publication functions\model_outputs_dev.csv', index = False)

### y_pred test

In [11]:
y_pred_test = get_outputs(final_ensemble_models_test, final_ensemble_scores_test, model_weights_test, len(test_BERTBase_OUS_400))

In [12]:
y_pred_emotions = index_to_emotion(y_pred_test)

In [13]:
pd.DataFrame(y_pred_emotions).to_csv('predictions_EMO.tsv', index = False, sep = '\t', header = None)